In [1]:
import os
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


/opt/anaconda3/envs/textC/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create directories for saving data and model
os.makedirs("./data", exist_ok=True)
os.makedirs("./model", exist_ok=True)

In [4]:
# Load the IMDB dataset and save locally
dataset = load_dataset("imdb")
dataset.save_to_disk("./data/imdb")

Saving the dataset (1/1 shards): 100%|██████████| 50000/50000 [00:00<00:00, 2594329.26 examples/s]


In [5]:
# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length', truncation=True, max_length=512)

In [6]:
# Tokenize and preprocess dataset
encoded_dataset = dataset.map(tokenize_data, batched=True)
encoded_dataset.save_to_disk("./data/tokenized_imdb")

Saving the dataset (1/1 shards): 100%|██████████| 50000/50000 [00:00<00:00, 102574.00 examples/s]


In [7]:
# Split dataset
train_dataset = encoded_dataset['train']
test_dataset = encoded_dataset['test']


In [2]:
# Define the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./model/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./model/logs",
    logging_steps=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True
)

/opt/anaconda3/envs/textC/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [4]:
# Define compute metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [5]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train and evaluate
trainer.train()
results = trainer.evaluate()
print(results)


NameError: name 'train_dataset' is not defined

In [ ]:
# Save the trained model locally
model.save_pretrained("./model/sentiment_model")

# Load model for inference
sentiment_pipeline = pipeline("text-classification", model="./model/sentiment_model")
result = sentiment_pipeline("I love this movie!")
print(result)  # [{'label': 'POSITIVE', 'score': 0.99}]
